In [1]:
import sys
sys.path.append("../") 

In [2]:
import torch
from transformers import AutoTokenizer
from models.transformer_lm import MiniTransformerLM
import random
import os

In [3]:
# Parámetros
device = "cuda" if torch.cuda.is_available() else "cpu"
vocab_model = "gpt2"
model_path = "../checkpoints/minitransformer.pt"
max_tokens = 128
num_samples = 5

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(vocab_model)
tokenizer.pad_token = tokenizer.eos_token

# Modelo
model = MiniTransformerLM(
    vocab_size=tokenizer.vocab_size,
    d_model=256,
    n_heads=4,
    n_layers=4,
    max_len=max_tokens
).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


MiniTransformerLM(
  (token_emb): Embedding(50257, 256)
  (pos_emb): Embedding(128, 256)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=256, out_features=50257, bias=True)
)

In [15]:
# Función de generación autoregresiva
import torch.nn.functional as F

def generar_texto(prompt, max_len=100, top_k=50, temperature=1.0):
    model.eval()

    # Agregar token por defecto si el prompt está vacío
    if not prompt.strip():
        prompt = tokenizer.eos_token  

    ids = tokenizer.encode(prompt, return_tensors="pt").to(device).long()
    attn_mask = torch.ones_like(ids)

    for _ in range(max_len):
        with torch.no_grad():
            logits = model(ids, attention_mask=attn_mask)
        next_logits = logits[:, -1, :] / temperature
        top_k_probs, top_k_indices = torch.topk(next_logits, k=top_k, dim=-1)
        probs = torch.nn.functional.softmax(top_k_probs, dim=-1)
        next_token = top_k_indices[0, torch.multinomial(probs, num_samples=1)]
        next_token = next_token.view(1, 1) 
        ids = torch.cat([ids, next_token], dim=1)
        attn_mask = torch.ones_like(ids)

    return tokenizer.decode(ids[0], skip_special_tokens=True)

# Métricas de diversidad
def distinct_n(seqs, n):
    total_ngrams = 0
    unique_ngrams = set()
    for text in seqs:
        tokens = text.split()
        ngrams = zip(*[tokens[i:] for i in range(n)])
        ngrams = list(ngrams)
        total_ngrams += len(ngrams)
        unique_ngrams.update(ngrams)
    return len(unique_ngrams) / total_ngrams if total_ngrams > 0 else 0



In [16]:
# Parámetros de generación
num_samples = 5
max_tokens = 100
top_k = 50
temperature = 1.0

# Generar muestras
resultados = []
for _ in range(num_samples):
    texto = generar_texto("", max_len=max_tokens, top_k=top_k, temperature=temperature)
    print("-" * 30)
    print(texto)
    resultados.append(texto)

# Calcular diversidad
d1 = distinct_n(resultados, 1)
d2 = distinct_n(resultados, 2)
print("\nMétricas de diversidad:")
print(f"Distinct-1: {d1:.4f}")
print(f"Distinct-2: {d2:.4f}")


------------------------------
, to and had The was big  you. very with so the mom to you. " a was.. had. is up were that was was in said a, she the they to the. the they The the she.. wanted him a they. time you,. to saw,.., his they, to their and said He to,. the. They. was to was a,." said " a that day was and They.. so a they to and time
------------------------------
.., They the. he. He " day with little she. to with He was's the They, not and. a The,. a in. so They.. it.." and and she her to was, and." were of the that a and his mom and. was was you. their they day her they she " The had. " and. for not, , She,. to said a it wanted and to He it. that, up She a He
------------------------------
 they little., The the and his the He and the to the's was day it time " little to, The.. to. to she time. were happy she a.. to, was little the on he the very was they not you and, time and was " The.'s and.. a with she. the saw Lily, their for She a " she, you, so she the. with I and The